
## How to Use the Notebooks

To run these notebooks, follow these steps:


### 1. Install **`gdal`** Package
Installing `gdal` via `pip` may fail. Therefore, we recommend using conda to install it:
```bash
conda install gdal -c conda-forge
```

### 2. Install **`polsartools`** Package

Before running the notebooks, you'll need to install the core **`polsartools`** package. 

**Option A:** Install from PyPI `pip` (stable release)

```bash
pip install polsartools
```

**Option B:** Install from conda (stable release)

```bash
conda install polsartools -c bnarayanarao
```


**Option C:**  Install the latest version from GitHub (if you need recent updates or fixes)

```bash
pip install git+https://github.com/Narayana-Rao/polsartools.git#egg=polsartools
```
> **Note:** If you encounter a `"module not found"` error or require newer features, prefer the GitHub installation (Option C).

### 3. Verify the Installation

After successfully installing `polsartools`, you can verify it by importing the package:

```bash
import polsartools as pst
```

If this runs without errors, you're ready to explore and run the notebooks.




In [ ]:
""" 

import required packages

"""
import os
import matplotlib.pyplot as plt
import numpy as np
import polsartools as pst
    

In [ ]:
""" 

Function to display images

"""

def plot_images(*filenames, lower_percentile=2, upper_percentile=98, max_cols=4):
    def compute_vmin_vmax(data):
        data_no_nan = data[np.isfinite(data)]
        vmin = np.percentile(data_no_nan, lower_percentile)
        vmax = np.percentile(data_no_nan, upper_percentile)
        return vmin, vmax

    num_images = len(filenames)

    if num_images == 1:
        # Single image case
        data = pst.read_rst(filenames[0])
        data[data==0]=np.nan
        vmin, vmax = compute_vmin_vmax(data)

        fig, ax = plt.subplots(figsize=(6, 6))
        im = ax.imshow(data, vmin=vmin, vmax=vmax)
        ax.set_title(os.path.basename(filenames[0]))
        fig.colorbar(im, ax=ax)
        plt.tight_layout()
        plt.show()
    else:
        # Multiple image case
        ncols = min(max_cols, num_images)
        nrows = int(np.ceil(num_images / ncols))

        fig, axes = plt.subplots(nrows, ncols, figsize=(5 * ncols, 5 * nrows))
        axes = np.array(axes).reshape(-1)  # Flatten in case it's a 2D grid

        for i, filename in enumerate(filenames):
            data = pst.read_rst(filename)
            data[data==0]=np.nan
            vmin, vmax = compute_vmin_vmax(data)
            ax = axes[i]
            im = ax.imshow(data, vmin=vmin, vmax=vmax)
            ax.set_title(os.path.basename(filename))
            fig.colorbar(im, ax=ax)

        for j in range(num_images, len(axes)):
            axes[j].axis('off')

        plt.tight_layout()
        plt.show()


#### 1. Extract multilooked C2 matrix from ALOS-2 FBD data
##### using **`polsartools.alos2_fbd_l11`** function, [click here for documentation](https://polsartools.readthedocs.io/en/latest/files/sensors/alos2.html#fbd-l1-1-alos2-fbd-l11c) or run `print(polsartools.alos2_fbd_l11.__.doc__)`


In [ ]:
inFolder =r"ALOS2_DP/ALOS2214806690-180515"


# The following function extracts a multilooked C2 matrix from ALOS-2 FBD data. 
# Provide path to the extracted folder

pst.alos2_fbd_l11(inFolder,azlks=8,rglks=4)

#### 2. Speckle filtering using polarimetric refined-Lee filter

In [ ]:
c2Folder = os.path.join(r'ALOS2_DP/ALOS2214806690-180515/ALOS2214806690-180515-FBDR','C2')
window_size = 3
pst.rlee(c2Folder,window_size=window_size)

#### 3. Generate a quick look RGB from C2 matrix

In [ ]:

c2Folder = os.path.join(r'ALOS2_DP/ALOS2214806690-180515/ALOS2214806690-180515-FBDR','rlee_3x3','C2')
# pst.dxpRGB(c2Folder,type=1)
# pst.dxpRGB(c2Folder,type=2)
# pst.dxpRGB(c2Folder,type=3)
pst.dxpRGB(c2Folder,type=4)

#### 4. Generating some polarimetric parameters from C2 matrix

##### 4.1 Barakat Degree of polarization 
##### using `polsartools.dopdp` function, [click here for documentation](https://polsartools.readthedocs.io/en/latest/files/functions/dual_pol/DOP_dp.html) or run `print(polsartools.dopdp.__.doc__)`


In [ ]:

pst.dopdp(c2Folder)


##### Preview the output

In [ ]:
plot_images(c2Folder+'/dopdp.tif', lower_percentile=2, upper_percentile=98, max_cols=4)



##### 4.2. Entropy-Alpha (H/α) parameters 
##### using **`polsartools.halphadp`** function, [click here for documentation](https://polsartools.readthedocs.io/en/latest/files/functions/dual_pol/halphadxp.html) or run `print(polsartools.halphadp.__.doc__)`


In [ ]:
pst.halphadp(c2Folder)


##### Preview the output

In [ ]:

plot_images(c2Folder + '/Hdp.tif',
            c2Folder + '/alphadp.tif', 
            c2Folder + '/e1_norm.tif', 
            c2Folder + '/e2_norm.tif', 
            lower_percentile=2, upper_percentile=98, max_cols=4)


##### Plot Entropy-Alpha plot for dual-pol
##### using **`polsartools.halpha_plot_dp`** function, [click here for documentation](https://polsartools.readthedocs.io/en/latest/files/functions/dual_pol/halpha_plot_dp.html) or run `print(polsartools.halpha_plot_dp.__.doc__)`

In [ ]:
hFile = c2Folder + '/Hdp.tif'
alphaFile = c2Folder + '/alphadp.tif'

pst.halpha_plot_dp(hFile,alphaFile)

##### 4.3. Shannon Entropy parameters (SE is the sum of contributions from intensity (SEI) and polarimetry (SEP))
##### using **`polsartools.shannon_h_dp`** function, [click here for documentation](https://polsartools.readthedocs.io/en/latest/files/functions/dual_pol/shannon_h_dp.html) or run `print(polsartools.shannon_h_dp.__.doc__)`

In [ ]:
pst.shannon_h_dp(c2Folder,window_size = 3)

##### Preview the output

In [ ]:

plot_images(c2Folder + '/H_Shannon.tif',
            c2Folder + '/HI_Shannon.tif',
            c2Folder + '/HP_Shannon.tif',
            lower_percentile=2, upper_percentile=98, max_cols=4)



##### 4.4. vegetation indices, Radar vegetation index (RVI), Dual-pol Radar Vegetation Index (DpRVI)

##### using **`polsartools.rvidp`** function, [click here for documentation](https://polsartools.readthedocs.io/en/latest/files/functions/dual_pol/RVI_dp.html) or run `print(polsartools.rvidp.__.doc__)`

##### using **`polsartools.dprvi`** function, [click here for documentation](https://polsartools.readthedocs.io/en/latest/files/functions/dual_pol/RVI_dp.html) or run `print(polsartools.dprvi.__.doc__)`




In [ ]:

pst.rvidp(c2Folder)

pst.dprvi(c2Folder)


##### Preview the output

In [ ]:


plot_images(c2Folder + '/rvidp.tif',
            c2Folder + '/dprvi.tif',
            lower_percentile=2, upper_percentile=98, max_cols=4)

